In [108]:
import pandas as pd 
import numpy as np 
import re
import nltk

In [3]:
df = pd.read_csv("../Data/resume_dataset.csv")
print(df.head())

   ID Category                                             Resume
0   1       HR  b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...
1   2       HR  b'Name Surname\nAddress\nMobile No/Email\nPERS...
2   3       HR  b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3   4       HR  b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4   5       HR  b"HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...


In [107]:
x = eval(df.Resume[9]).decode("utf")
split_text = x.split("\n")


In [119]:
use_data =list(map(nltk.word_tokenize,split_text))

In [123]:
use_data = list(map(nltk.pos_tag,use_data))

In [124]:
use_data

[[('Human', 'NNP'),
  ('Resources', 'NNP'),
  ('Coordinator', 'NNP'),
  ('Resume', 'NNP')],
 [('Example', 'NN')],
 [],
 [('Steven', 'NNP'), ('B.', 'NNP'), ('Manning', 'NNP')],
 [('3249', 'CD'), ('Oral', 'NNP'), ('Lake', 'NNP'), ('Road', 'NNP')],
 [('Minneapolis', 'NNP'), (',', ','), ('MN', 'NNP'), ('55401', 'CD')],
 [('Phone', 'NN'), (':', ':'), ('952-400-4765', 'JJ')],
 [('Email', 'NN'),
  (':', ':'),
  ('sbmanning', 'NN'),
  ('@', 'NN'),
  ('anymail.com', 'NN')],
 [('Career', 'NNP'), ('Objective', 'NNP'), (':', ':')],
 [('To', 'TO'),
  ('work', 'VB'),
  ('as', 'IN'),
  ('a', 'DT'),
  ('HR', 'NNP'),
  ('coordinator', 'NN'),
  ('with', 'IN'),
  ('“', 'NNP'),
  ('Noble', 'NNP'),
  ('Hospital', 'NNP'),
  ('”', 'NNP'),
  ('and', 'CC'),
  ('apply', 'VB'),
  ('my', 'PRP$'),
  ('skills', 'NNS'),
  ('and', 'CC'),
  ('experience', 'NN'),
  ('in', 'IN')],
 [('enhancing', 'VBG'),
  ('healthcare', 'NN'),
  ('for', 'IN'),
  ('patients', 'NNS'),
  ('and', 'CC'),
  ('retaining', 'VBG'),
  ('sincere'

In [ ]:
f = open("try.txt","w")
f.write(x)

In [102]:
v = re.match(r"\S+.@.\S+",x,re.I)

In [103]:
v.group()

AttributeError: 'NoneType' object has no attribute 'group'

In [79]:
x

"Human Resources Coordinator Resume\nExample\n\nSteven B. Manning\n3249 Oral Lake Road\nMinneapolis, MN 55401\nPhone: 952-400-4765\nEmail: sbmanning@anymail.com\nCareer Objective:\nTo work as a HR coordinator with “Noble Hospital” and apply my skills and experience in\nenhancing healthcare for patients and retaining sincere and hard-working employees.\nSummary of Skills:\n\uf0b7Strong knowledge of the functioning and procedures of human resources\ndepartment\n\uf0b7Experience in maintaining smooth relationships with management, employees,\nvendors, and contractors\n\uf0b7Excellent consulting, communication, interpersonal, and computer skills\n\uf0b7Ability to negotiate salary with employees and meet deadlines\n\uf0b7Excellent conflict management skills and proficiency in developing and\nimplementing strategies\n\uf0b7Familiarity with medical environment and legal codes and regulations associated\nwith it\nWork Experience:\nHR Coordinator\nPQR Medical Center, Minneapolis, MN\nAugust 201